In [ ]:
# -*- coding: utf-8 -*-
"""
// ストリーマ放電の超簡単コード
// Copyright: Atsushi Komuro 
// 最終更新 2025/8/27
// C言語で作成したものをpythonに移植しました。
// 再配布はしないようにお願いします。(間違いがある可能性があるため)

Updates vs previous version:
- Full 2D fields saved as .npy files under outputdata_npy/ (every 100 steps).
- Axis line (i=1) time-series saved per-variable into single appendable .npy memmaps (output_axis/):
  ne, O2p, N2p, Om, O2m, phi, Ey + time and step arrays.
- Robust to interruption: axis_meta.json tracks the next write index; files are memmaps and flushed each step.

NOTE:
- For appendable .npy we use numpy.lib.format.open_memmap with a fixed maximum length (MAX_STEPS).
- If you need more than MAX_STEPS, increase it here before running (or create a new folder).
"""

import os
import json
import math
import numpy as np
from numpy.lib import format as npformat

# ---------------------- Constants & Parameters ----------------------
NR    = 200          # 横方向の格子数
NZ    = 100          # 縦方向の格子数
delta = 10e-6        # 格子間隔 [m]

T0 = 300.0           # 初期ガス温度 [K]

Avogadro = 6.022141e+23         # [1/mol]
GasR     = 0.082057366080960     # [L atm K^-1 mol^-1]
QELEC    = 1.602176462e-19       # [C]
MOL      = (Avogadro*1000.0)/(GasR*T0)  # 分子数(const)
E0       = 8.85e-12              # [F/m]=[C/V/m]

pO2 = 0.2
pN2 = 0.8

# Geometry (needle-plane)
bgap = NZ * delta                # ギャップ長
cr   = 300e-6                    # 先端曲率半径

# Simulation control
DT       = 1.0e-12
NSTEPS   = 5001                  # 0..5000
SAVE_EVERY = 100                 # full 2D save cadence

# Output dirs
OUT2D_DIR = "outputdata_npy"
AXIS_DIR  = "output_axis"

# Axis-logging capacity (memmap length). Should be >= NSTEPS to avoid truncation.
MAX_STEPS = NSTEPS

# ---------------------- Helpers (memory) ------------
def zeros_vec(n):
    return np.zeros(n, dtype=float)
def zeros_mat(n1, n2):
    return np.zeros((n1, n2), dtype=float)

# ---------------------- Hyperbolic electrode (analytic Laplace) ----------
def hyperbolic_curve(Vele, cr, b, r, z):
    a = math.sqrt(cr*b)
    if (z/b)**2 - (r/a)**2 >= 1:
        return Vele
    elif z == 0.0:
        return 0.0
    else:
        thetaI = math.atan2(a, b)
        Q = math.sqrt(b*b + a*a)
        term = (r*r + z*z - Q*Q)
        inside = term*term - 4.0*(-r*r*Q*Q)
        a2 = (-(r*r + z*z - Q*Q) + math.sqrt(inside))/2.0
        a2 = math.sqrt(max(a2, 0.0))
        b2 = math.sqrt(max(Q*Q - a2*a2, 0.0))
        theta = math.atan2(a2, b2)
        denom = math.log(1.0/(math.tan(thetaI/2.0)))
        if denom == 0.0:
            return 0.0
        V = Vele * (math.log(1.0/math.tan(theta/2.0))) / denom
        return V

# ---------------------- Poisson discretization & solver ----------
def discretization(N, M, rh, zh, P1, P2, P3, P4, P5):
    h1 = zeros_mat(N, M)
    h2 = zeros_mat(N, M)
    h3 = zeros_mat(N, M)
    h4 = zeros_mat(N, M)

    for i in range(N):
        for j in range(M):
            if i == 0:
                if j == M-1:
                    h1[i, j] = h1[i, j-1] * 0.5
                else:
                    h1[i, j] = zh[j+1] - zh[j]
                h2[i, j] = rh[i]
                h3[i, j] = rh[i+1] - rh[i]
                if j == 0:
                    h4[i, j] = zh[j]
                else:
                    h4[i, j] = zh[j] - zh[j-1]
            elif j == 0:
                h1[i, j] = zh[j+1] - zh[j]
                h2[i, j] = rh[i] - rh[i-1]
                if i == N-1:
                    h3[i, j] = h3[i-1, j]
                else:
                    h3[i, j] = rh[i+1] - rh[i]
                h4[i, j] = zh[j]
            elif j == M-1:
                h1[i, j] = h1[i, j-1] * 0.5
                h2[i, j] = rh[i] - rh[i-1]
                if i == N-1:
                    h3[i, j] = h3[i-1, j]
                else:
                    h3[i, j] = rh[i+1] - rh[i]
                h4[i, j] = zh[j] - zh[j-1]
            elif i == N-1:
                if j == M-1:
                    h1[i, j] = h1[i, j-1] * 0.5
                else:
                    h1[i, j] = zh[j+1] - zh[j]
                h2[i, j] = rh[i] - rh[i-1]
                h3[i, j] = h3[i-1, j]
                h4[i, j] = zh[j] - zh[j-1]
            else:
                h1[i, j] = zh[j+1] - zh[j]
                h2[i, j] = rh[i] - rh[i-1]
                h3[i, j] = rh[i+1] - rh[i]
                h4[i, j] = zh[j] - zh[j-1]

    for i in range(N):
        for j in range(M):
            hh1 = h1[i, j]; hh2 = h2[i, j]; hh3 = h3[i, j]; hh4 = h4[i, j]
            r0_double = 2.0 * rh[i]
            if i == 0:
                P1[i, j] = (hh1*hh4*hh3*hh3) / (hh3*hh3 + 2.0*hh1*hh4)
                P2[i, j] = -1.0 / (hh1*(hh1+hh4))
                P3[i, j] = 0.0
                P4[i, j] = -2.0 / (hh3*hh3)
                P5[i, j] = -1.0 / (hh4*(hh1+hh4))
            else:
                P1[i, j] = hh1*hh2*hh3*hh4 / (r0_double*hh2*hh3 + (r0_double + hh2 - hh3)*hh1*hh4)
                P2[i, j] = -r0_double / (hh1*(hh1+hh4))
                P3[i, j] = (-r0_double + hh3) / (hh2*(hh2+hh3))
                P4[i, j] = -(r0_double + hh2) / (hh3*(hh2+hh3))
                P5[i, j] = -r0_double / (hh4*(hh1+hh4))

def poisson_calc(V, phi, rho, rh, zh, r, z, NR, NZ, P1, P2, P3, P4, P5):
    loop   = 0
    MaxPhi = 1.0e-10
    Conv   = 1.0e-3

    while True:
        if loop != 0 and (loop % 20000 == 0):
            print(f"{loop:05d}  {MaxPhi:e}")
        MaxErr = 0.0
        for i in range(NR):
            for j in range(NZ):
                Prev_phi = phi[i, j]
                if i == 0:
                    if j == 0:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*phi[i, j+1] - P4[i, j]*phi[i+1, j] - P5[i, j]*0.0)
                    elif j == NZ-1:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*0.0       - P4[i, j]*phi[i+1, j] - P5[i, j]*phi[i, j-1])
                    else:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*phi[i, j+1] - P4[i, j]*phi[i+1, j] - P5[i, j]*phi[i, j-1])
                elif j == 0:
                    if i == NR-1:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*phi[i, j+1] - P3[i, j]*phi[i-1, j] - P4[i, j]*phi[i, j] - P5[i, j]*0.0)
                    else:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*phi[i, j+1] - P3[i, j]*phi[i-1, j] - P4[i, j]*phi[i+1, j] - P5[i, j]*0.0)
                elif j == NZ-1:
                    if i == NR-1:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*0.0       - P3[i, j]*phi[i-1, j] - P4[i, j]*phi[i, j] - P5[i, j]*phi[i, j-1])
                    else:
                        phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*0.0       - P3[i, j]*phi[i-1, j] - P4[i, j]*phi[i+1, j] - P5[i, j]*phi[i, j-1])
                elif i == NR-1:
                    phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*phi[i, j+1] - P3[i, j]*phi[i-1, j] - P4[i, j]*phi[i, j] - P5[i, j]*phi[i, j-1])
                else:
                    phi[i, j] = P1[i, j]*(rho[i, j] - P2[i, j]*phi[i, j+1] - P3[i, j]*phi[i-1, j] - P4[i, j]*phi[i+1, j] - P5[i, j]*phi[i, j-1])

                if abs(phi[i, j]) > MaxPhi:
                    MaxPhi = abs(phi[i, j])
                CurErr = abs(phi[i, j] - Prev_phi) / MaxPhi
                if CurErr > MaxErr:
                    MaxErr = CurErr
        if loop % 100 == 0:
            print(loop,MaxErr)
        loop += 1
        if MaxErr <= Conv:
            break

# ---------------------- Electron advection (upwind) ----------
def nele_flow(nr, nz, M, vr, vz, dt, Vol, Sr, Sz):
    dSz = zeros_mat(nr, nz)
    dSr = zeros_mat(nr, nz)

    j = 0
    for i in range(nr):
        if vz[i, j] >= 0.0:
            dSz[i, j] = dt * vz[i, j] * M[i, j] * Sz[i, j]
        else:
            dSz[i, j] = dt * vz[i, j] * M[i, j] * Sz[i, j]

    for i in range(nr):
        for j in range(1, nz):
            if vz[i, j] >= 0.0:
                dSz[i, j] = dt * vz[i, j] * M[i, j-1] * Sz[i, j]
            else:
                dSz[i, j] = dt * vz[i, j] * M[i, j]   * Sz[i, j]

    for i in range(nr):
        for j in range(nz):
            if i == 0:
                dSr[i, j] = 0.0
            else:
                if vr[i, j] >= 0.0:
                    dSr[i, j] = dt * vr[i, j] * M[i-1, j] * Sr[i, j]
                else:
                    dSr[i, j] = dt * vr[i, j] * M[i, j]   * Sr[i, j]

    for i in range(nr-1):
        for j in range(nz-1):
            M[i, j] += (dSz[i, j] - dSz[i, j+1] + dSr[i, j] - dSr[i+1, j]) / Vol[i, j]  # NOTE: using Sz as cross-section to match cylindrical volume discretization

    j = nz-1
    for i in range(nr-1):
        M[i, j] += (dSz[i, j] - dSz[i, j] + dSr[i, j] - dSr[i+1, j]) / Vol[i, j]
    i = nr-1
    for j in range(nz-1):
        M[i, j] += (dSz[i, j] - dSz[i, j+1] + dSr[i, j] - dSr[i, j]) / Vol[i, j]
    i = nr-1; j = nz-1
    M[i, j] += (dSz[i, j] - dSz[i, j] + dSr[i, j] - dSr[i, j]) / Vol[i, j]

# ---------------------- Axis memmap append utilities ----------------------
def _open_or_create_axis_memmaps(nz, max_steps, outdir):
    """
    Returns dict of memmaps: vars: ne,O2p,N2p,Om,O2m,phi,Ey; plus time, step.
    Also returns meta dict with 'next_index'.
    """
    os.makedirs(outdir, exist_ok=True)
    meta_path = os.path.join(outdir, "axis_meta.json")

    # Files to open/create
    var_names = ["ne","O2p","N2p","Om","O2m","phi","Ey"]
    mmap = {}
    created = False

    # Helper to create memmap with NaN fill
    def create_map(path, shape):
        mm = npformat.open_memmap(path, mode="w+", dtype=np.float64, shape=shape)
        mm[:] = np.nan
        mm.flush()
        return mm

    # Open/create per-variable arrays
    for v in var_names:
        path = os.path.join(outdir, f"{v}_axis.npy")
        if os.path.exists(path):
            mm = npformat.open_memmap(path, mode="r+")
            # sanity: shape
            if mm.shape != (max_steps, nz):
                raise ValueError(f"Shape mismatch for {path}: got {mm.shape}, expected {(max_steps, nz)}")
        else:
            mm = create_map(path, (max_steps, nz))
            created = True
        mmap[v] = mm

    # time and step index arrays
    time_path = os.path.join(outdir, "time.npy")
    step_path = os.path.join(outdir, "step.npy")
    if os.path.exists(time_path):
        tmm = npformat.open_memmap(time_path, mode="r+")
        if tmm.shape != (max_steps,):
            raise ValueError(f"Shape mismatch for time.npy: {tmm.shape} != {(max_steps,)}")
    else:
        tmm = create_map(time_path, (max_steps,))
        created = True

    if os.path.exists(step_path):
        smm = npformat.open_memmap(step_path, mode="r+")
        if smm.shape != (max_steps,):
            raise ValueError(f"Shape mismatch for step.npy: {smm.shape} != {(max_steps,)}")
    else:
        smm = create_map(step_path, (max_steps,))
        created = True

    mmap["time"] = tmm
    mmap["step"] = smm

    # meta
    if os.path.exists(meta_path):
        with open(meta_path, "r", encoding="utf-8") as f:
            meta = json.load(f)
        next_index = int(meta.get("next_index", 0))
    else:
        next_index = 0

    meta = {"next_index": next_index, "max_steps": max_steps, "nz": nz}
    return mmap, meta, meta_path

def _append_axis_row(mmap, meta, meta_path, irow, arrays, time_val, step_val):
    """Append one row (axis line) for each variable at index 'irow'."""
    if irow >= meta["max_steps"]:
        raise RuntimeError(f"Axis logs full: irow={irow} >= MAX_STEPS={meta['max_steps']}")
    # arrays dict expected keys: "ne","O2p","N2p","Om","O2m","phi","Ey"
    for k, v in arrays.items():
        mmap[k][irow, :] = v  # shape (NZ,)
        mmap[k].flush()
    mmap["time"][irow] = time_val
    mmap["time"].flush()
    mmap["step"][irow] = step_val
    mmap["step"].flush()

    meta["next_index"] = irow + 1
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f)

# ---------------------- Main simulation ----------------------
def main():
    a = math.sqrt(cr * bgap)

    # Allocate arrays
    r   = zeros_vec(NR); z   = zeros_vec(NZ)
    rh  = zeros_vec(NR); zh  = zeros_vec(NZ)
    phi = zeros_mat(NR, NZ)
    Ex  = zeros_mat(NR, NZ); Ey  = zeros_mat(NR, NZ); absE = zeros_mat(NR, NZ)

    N2  = zeros_mat(NR, NZ); O2  = zeros_mat(NR, NZ)
    N2p = zeros_mat(NR, NZ); O2p = zeros_mat(NR, NZ)
    Om  = zeros_mat(NR, NZ); O2m = zeros_mat(NR, NZ)
    ne  = zeros_mat(NR, NZ); rho = zeros_mat(NR, NZ)

    vex = zeros_mat(NR, NZ); vey = zeros_mat(NR, NZ)
    k1  = zeros_mat(NR, NZ); k2  = zeros_mat(NR, NZ); k3 = zeros_mat(NR, NZ)

    dr = zeros_vec(NR); dz = zeros_vec(NZ)
    Vol = zeros_mat(NR, NZ); Sr = zeros_mat(NR, NZ); Sz = zeros_mat(NR, NZ)

    P1 = zeros_mat(NR, NZ); P2 = zeros_mat(NR, NZ); P3 = zeros_mat(NR, NZ)
    P4 = zeros_mat(NR, NZ); P5 = zeros_mat(NR, NZ)
    lphi = zeros_mat(NR, NZ); pphi = zeros_mat(NR, NZ)

    # Mesh
    for i in range(NR):
        r[i] = delta * float(i)
    for j in range(NZ):
        z[j] = delta * float(j)

    for i in range(NR-1):
        rh[i] = 0.5*(r[i] + r[i+1])
    i = NR-1
    rh[i] = 0.5*(r[i] + (r[i] + delta))

    for j in range(NZ-1):
        zh[j] = 0.5*(z[j] + z[j+1])
    j = NZ-1
    zh[j] = 0.5*(z[j] + (z[j] + delta))

    for i in range(NR-1):
        dr[i] = r[i+1] - r[i]
    for j in range(NZ-1):
        dz[j] = z[j+1] - z[j]
    dr[NR-1] = dr[NR-2]
    dz[NZ-1] = dz[NZ-2]

    for i in range(NR):
        for j in range(NZ):
            if i == NR-1:
                Vol[i, j] = dz[j]*(r[i]+r[i])*dr[i]/2.0
                Sr[i, j]  = r[i]*dz[j]
                Sz[i, j]  = (r[i]+r[i])*dr[i]/2.0
            else:
                Vol[i, j] = dz[j]*(r[i]+r[i+1])*dr[i]/2.0
                Sr[i, j]  = r[i]*dz[j]
                Sz[i, j]  = (r[i]+r[i+1])*dr[i]/2.0

    discretization(NR, NZ, rh, zh, P1, P2, P3, P4, P5)

    V  = 4000.0
    dt = DT

    # Initial particles
    nmax = 1e19
    z0 = 900.0e-6
    sr0 = 100e-6
    sz0 = 100e-6

    for i in range(NR):
        for j in range(NZ):
            N2[i, j]  = pN2*MOL
            O2[i, j]  = pO2*MOL
            gauss = math.exp(-(rh[i]*rh[i])/(2.0*sr0*sr0) - ((zh[j]-z0)*(zh[j]-z0))/(2.0*sz0*sz0))
            N2p[i, j] = pN2*nmax*gauss
            O2p[i, j] = pO2*nmax*gauss
            ne[i, j]  = N2p[i, j] + O2p[i, j]
            O2m[i, j] = 0.0
            Om[i, j]  = 0.0

    # Prepare output dirs
    os.makedirs(OUT2D_DIR, exist_ok=True)
    os.makedirs(AXIS_DIR, exist_ok=True)

    # Save coordinates (1D) once as .npy
    np.save(os.path.join(OUT2D_DIR, "rh.npy"), rh)
    np.save(os.path.join(OUT2D_DIR, "zh.npy"), zh)

    # Axis memmaps (appendable)
    axis_mmap, axis_meta, axis_meta_path = _open_or_create_axis_memmaps(NZ, MAX_STEPS, AXIS_DIR)
    next_idx = axis_meta.get("next_index", 0)

    time = 0.0

    for step in range(0, NSTEPS):
        # Charge density
        for i in range(NR):
            for j in range(NZ):
                CPp = N2p[i, j] + O2p[i, j]
                CPm = Om[i, j] + O2m[i, j] + ne[i, j]
                if i == 0:
                    rho[i, j] = 0.5*QELEC/E0*(CPp - CPm)
                else:
                    rho[i, j] = rh[i]*QELEC/E0*(CPp - CPm)

        # Laplace potential
        for i in range(NR):
            for j in range(NZ):
                lphi[i, j] = hyperbolic_curve(V, cr, bgap, rh[i], zh[j])

        # Poisson field
        poisson_calc(V, pphi, rho, rh, zh, r, z, NR, NZ, P1, P2, P3, P4, P5)

        # Total potential
        for i in range(NR):
            for j in range(NZ):
                phi[i, j] = pphi[i, j] + lphi[i, j]

        # Electric field components & magnitude
        for i in range(NR):
            for j in range(NZ):
                if i == 0:
                    if j == 0:
                        Ex[i, j] = -0.0
                        Ey[i, j] = -(phi[i, j] - 0.0)/ (zh[0]) / (MOL*1e-21)
                        absE[i, j] = math.sqrt(Ex[i, j]*Ex[i, j] + Ey[i, j]*Ey[i, j])
                    else:
                        Ex[i, j] = -0.0
                        Ey[i, j] = -(phi[i, j] - phi[i, j-1])/(zh[j]-zh[j-1])/(MOL*1e-21)
                        absE[i, j] = math.sqrt(Ex[i, j]*Ex[i, j] + Ey[i, j]*Ey[i, j])
                elif j == 0:
                    Ex[i, j] = -(phi[i, j] - phi[i-1, j])/(rh[i]-rh[i-1])/(MOL*1e-21)
                    Ey[i, j] = -(phi[i, j] - 0.0)/ (zh[0]) / (MOL*1e-21)
                    absE[i, j] = math.sqrt(Ex[i, j]*Ex[i, j] + Ey[i, j]*Ey[i, j])
                else:
                    Ex[i, j] = -(phi[i, j] - phi[i-1, j])/(rh[i]-rh[i-1])/(MOL*1e-21)
                    Ey[i, j] = -(phi[i, j] - phi[i, j-1])/(zh[j]-zh[j-1])/(MOL*1e-21)
                    absE[i, j] = math.sqrt(Ex[i, j]*Ex[i, j] + Ey[i, j]*Ey[i, j])

        # Transport coefficients
        for i in range(NR):
            for j in range(NZ):
                vex[i, j] = -3.74e24*(Ex[i, j]*1e-21)*pow(absE[i, j], -0.25)
                vey[i, j] = -3.74e24*(Ey[i, j]*1e-21)*pow(absE[i, j], -0.25)
                absv = math.sqrt(vex[i, j]*vex[i, j] + vey[i, j]*vey[i, j])
                k1[i, j] = 1.4e-20*math.exp(-(660.0/absE[i, j]))*absv
                k2[i, j] = (5.0)*6.0e-23*math.exp(-(100.0/absE[i, j]))*absv
                k3[i, j] = (5.0*5.0)*1.6e-47*pow(absE[i, j], -1.1)*absv

        # Advection (electrons only)
        nele_flow(NR, NZ, ne, vex, vey, dt, Vol, Sr, Sz)

        # Chemistry (Euler)
        bgion = 1e14
        for i in range(NR):
            for j in range(NZ):
                r1n = dt*k1[i, j]*N2[i, j]*ne[i, j]
                r1o = dt*k1[i, j]*O2[i, j]*ne[i, j]
                r2  = dt*k2[i, j]*O2[i, j]*ne[i, j]
                r3  = dt*k3[i, j]*O2[i, j]*O2[i, j]*ne[i, j]

                ne[i, j]  += r1n + r1o - r2 - r3 + bgion
                N2[i, j]  += -r1n
                O2[i, j]  += -r1o - r2 - r3
                N2p[i, j] += r1n + pN2*bgion
                O2p[i, j] += r1o + pO2*bgion
                Om[i, j]  += r2
                O2m[i, j] += r3

        # ---- Axis append (i=1) every step ----
        i_axis = 1
        axis_arrays = {
            "ne":  ne[i_axis, :].copy(),
            "O2p": O2p[i_axis, :].copy(),
            "N2p": N2p[i_axis, :].copy(),
            "Om":  Om[i_axis, :].copy(),
            "O2m": O2m[i_axis, :].copy(),
            "phi": phi[i_axis, :].copy(),
            "Ey":  Ey[i_axis, :].copy(),
        }
        _append_axis_row(axis_mmap, axis_meta, axis_meta_path, next_idx, axis_arrays, time, step)
        next_idx += 1

        # ---- Save full 2D fields as .npy every SAVE_EVERY steps ----
        if step % SAVE_EVERY == 0:
            np.save(os.path.join(OUT2D_DIR, f"ne_{step:05d}.npy"),  ne)
            np.save(os.path.join(OUT2D_DIR, f"absE_{step:05d}.npy"), absE)
            np.save(os.path.join(OUT2D_DIR, f"Ex_{step:05d}.npy"),  Ex)
            np.save(os.path.join(OUT2D_DIR, f"Ey_{step:05d}.npy"),  Ey)
            np.save(os.path.join(OUT2D_DIR, f"phi_{step:05d}.npy"), phi)
            print(f"step = {step}, Time = {time*1e9:.4f} ns (saved .npy)")
        print(f"step = {step}, Time = {time*1e9:.4f} ns")
        time += dt

if __name__ == "__main__":
    main()


0 0.0
step = 0, Time = 0.0000 ns (saved .npy)
step = 0, Time = 0.0000 ns
0 1.0
100 0.005788038715528257
200 0.002129469817488707
300 0.0011335492679928677
step = 1, Time = 0.0010 ns
0 0.023378130232657838
100 0.0024600252776917535
200 0.001153330364603664
step = 2, Time = 0.0020 ns
0 0.009114884602117753
100 0.0017217938098717106
step = 3, Time = 0.0030 ns
0 0.005732670277088607
100 0.001393714858546738
step = 4, Time = 0.0040 ns
0 0.004398006011843496
100 0.0012060976429028125
step = 5, Time = 0.0050 ns
0 0.0036737704700526546
100 0.0010869258954062273
step = 6, Time = 0.0060 ns
0 0.003213499866940161
100 0.0010030361595603705
step = 7, Time = 0.0070 ns
0 0.002891500526061853
step = 8, Time = 0.0080 ns
0 0.002653086120395774
step = 9, Time = 0.0090 ns
0 0.002470169021112622
step = 10, Time = 0.0100 ns
0 0.002323878415850454
step = 11, Time = 0.0110 ns
0 0.0022052442376500393
step = 12, Time = 0.0120 ns
0 0.002105305141294416
step = 13, Time = 0.0130 ns
0 0.0020216575126118515
step = 1

KeyboardInterrupt: 

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

axis_dir = Path("output_axis")
grid_dir = Path("outputdata_npy")

# --- 必須ファイル読み込み ---
ne_axis = np.load(axis_dir / "ne_axis.npy", mmap_mode="r")   # shape ~ (T_max, NZ)
Om_axis = np.load(axis_dir / "Om_axis.npy", mmap_mode="r")   # shape ~ (T_max, NZ)
O2m_axis = np.load(axis_dir / "O2m_axis.npy", mmap_mode="r")   # shape ~ (T_max, NZ)
O2p_axis = np.load(axis_dir / "O2p_axis.npy", mmap_mode="r")   # shape ~ (T_max, NZ)
N2p_axis = np.load(axis_dir / "N2p_axis.npy", mmap_mode="r")   # shape ~ (T_max, NZ)
time    = np.load(axis_dir / "time.npy", mmap_mode="r")      # shape (T_max,)
zh      = np.load(grid_dir / "zh.npy")                       # shape (NZ,)

chg = (N2p_axis+O2p_axis) - (ne_axis+Om_axis+O2m_axis)

# --- どこまで書けているか（途中停止対応） ---
meta_path = axis_dir / "axis_meta.json"
if meta_path.exists():
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    T = int(meta.get("next_index", 0))
else:
    # メタがなければ、有効な time の要素数で推定
    T = int(np.count_nonzero(np.isfinite(time)))

if T == 0:
    raise RuntimeError("有効な時系列データが見つかりません。シミュレーションを少なくとも1ステップ実行してください。")

# --- 軸（単位変換） ---
t_ns = time[:T] * 1e9   # s -> ns
z_mm = zh * 1e3         # m -> mm

# --- 等高線用のグリッド（X: 時間, Y: z） ---
TT, ZZ = np.meshgrid(t_ns, z_mm, indexing="xy")   # shape: (NZ, T)
Zdata = chg[:T, :].T                          # (NZ, T) に転置

# --- 描画 ---
plt.figure(figsize=(7, 4))
cs = plt.contourf(TT, ZZ, Zdata, levels=100)      # レベル数はお好みで
plt.xlabel("Time [ns]")
plt.ylabel("z [mm]")
plt.xlim([0, 2.5])
plt.c
cbar = plt.colorbar(cs)
cbar.set_label("n_e [m$^{-3}$]")
plt.tight_layout()
plt.show()


In [ ]:

en = len(Zdata[1,:])
plt.plot(Zdata[:,1])
plt.plot(Zdata[:,en-1])
plt.ylim([-0.1e20,0.3e20])